##### This script essentially uses the pre-trained SVM model to perform face detection on a test image using a sliding window approach at various scales. Detected bounding box coordinates are shown on the test image, providing visual representation of detected faces.

In [1]:
!git clone https://github.com/aspagnuolo/Face_detection.git

Cloning into 'Face_detection'...
remote: Enumerating objects: 7451, done.
remote: Total 7451 (delta 0), reused 0 (delta 0), pack-reused 7451
Receiving objects: 100% (7451/7451), 197.23 MiB | 18.86 MiB/s, done.
Updating files: 100% (7449/7449), done.


**1. Importing Required Modules and Libraries:**

* Import functions from a custom utility module.
* Import the joblib library for loading the pre-trained SVM model.
* Import necessary classes from scikit-learn for creating custom transformers.
* Import the Pipeline class for creating a processing pipeline.

In [2]:
from Face_detection.utils import *
import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

**2. Loading Pre-trained SVM Model:**

* Load a pre-trained SVM model that includes hard negatives. This model was trained using the script in _train_model.ipynb_.

In [3]:
svm_with_hard_negatives=joblib.load('/content/Face_detection/svm_with_hard_negatives')

**3. Creating a Custom Transformer for Object Detection:**

* ObjectDetectionTransformer class is defined, inheriting from BaseEstimator and TransformerMixin.
* The constructor (__init__) initializes the transformer with attributes including the pre-trained SVM model, window sizes, step size, downscale factor, and confidence threshold.
* detect_objects method takes an image and a window size, and performs object detection using a sliding window approach at different scales. It calculates HOG features for each window, predicts using the SVM model, and accumulates detections based on confidence scores.
* transform method takes input data (images) and returns detected bounding box coordinates after non-maximum suppression (see utils.py)

In [4]:
class ObjectDetectionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, svm_with_hard_negatives, winSizes, stepSize, downscale, threshold):
        self.svm_with_hard_negatives = svm_with_hard_negatives
        self.winSizes = winSizes
        self.stepSize = stepSize
        self.downscale = downscale
        self.threshold = threshold

    def detect_objects(self, image, winSize):
        detections = []
        for scale in np.linspace(1.0, self.downscale, 5)[::-1]:
            resized_image = cv2.resize(image, (int(image.shape[1] / scale), int(image.shape[0] / scale)))
            for (x, y, window) in sliding_window(resized_image, self.stepSize, winSize):
                if window.shape[0] != winSize[1] or window.shape[1] != winSize[0]:
                    continue
                features = get_hog_features(window)
                confidence = self.svm_with_hard_negatives.decision_function([features])[0]
                if confidence >= self.threshold:
                    x = int(x * scale)
                    y = int(y * scale)
                    w = int(winSize[0] * scale)
                    h = int(winSize[1] * scale)
                    detections.append((x, y, x + w, y + h))
        return detections

    def transform(self, X, y=None):
        all_detections = []
        for winSize in self.winSizes:
            detections = self.detect_objects(X, winSize)
            all_detections.extend(detections)
        nms_detections = non_max_suppression(np.array(all_detections), overlap_threshold=0.2)
        return nms_detections

**4. Defining Parameters for Object Detection:**

* winSizes: A list of window sizes (bounding box dimensions) for object detection.
* stepSize: Step size for sliding window approach.
* downscale: A factor for resizing the image to detect objects at different scales.
* threshold: Confidence threshold for deciding if an object is detected.

**5. Creating an Object Detection Pipeline:**

object_detection_pipeline is defined as a scikit-learn Pipeline with a single step using the ObjectDetectionTransformer. This pipeline encapsulates the object detection process.

In [5]:
winSizes = [(128, 128)]
stepSize = 10
downscale = 1.5
threshold = 0.7

object_detection_pipeline = Pipeline([
    ('object_detection', ObjectDetectionTransformer(svm_with_hard_negatives, winSizes, stepSize, downscale, threshold))
])

**6. Loading and Preparing a Test Image:**

* test_image_path: Path to the test image file.
* test_image: Load the test image using OpenCV.

**7. Performing Object Detection:**

nms_detections: Perform object detection using the defined pipeline on the test image.
If objects are detected, the bounding box coordinates are stored in the nms_detections array.

In [ ]:
test_image_path = "/content/Face_detection/test/thelastofus.jpg"
test_image = cv2.imread(test_image_path)

nms_detections = object_detection_pipeline.transform(test_image)

if len(nms_detections) > 0:
    bounding_box_list = nms_detections.tolist()
    print(bounding_box_list)
else:
    print("NO FACES DETECTED IN THE IMAGE")

**8. Displaying Results:**

* If no faces are detected, the script prints "NO FACES DETECTED IN THE IMAGE".
* If faces are detected:
  * The script creates a copy of the test image (result_image).
  * It draws green rectangles around the detected bounding boxes on result_image.
  * The modified image is displayed using OpenCV, showing the detected objects.

In [ ]:
from google.colab.patches import cv2_imshow
if len(nms_detections) == 0:
    print("NO FACES DETECTED IN THE IMAGE")
else:
    result_image = test_image.copy()
    for (x1, y1, x2, y2) in nms_detections:
        cv2.rectangle(result_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2_imshow(result_image)
